In [5]:
import os
import numpy as np
import pandas as pd
import extract_data as dt

In [6]:
# def npy_files_to_df(directory_path, batch_size=10, save_intermediate=True, output_path='output.csv'):
#     dir_path = Path(directory_path)
    
#     # Get all .npy files in the directory
#     npy_files = list(dir_path.glob('*.npy'))
    
#     if not npy_files:
#         raise ValueError(f"No .npy files found in {directory_path}")
    
#     # Process files in batches
#     for i in range(0, len(npy_files), batch_size):
#         batch_files = npy_files[i:i + batch_size]
#         batch_data = []
        
#         print(f"Processing batch {i//batch_size + 1}/{(len(npy_files)-1)//batch_size + 1}")
        
#         # Process each file in the current batch
#         for file_path in batch_files:
#             try:
#                 # Load and process one file at a time
#                 data = np.load(file_path)
                
#                 # Get shape information before flattening
#                 shape_info = data.shape
                
#                 # Create row dict with filename and shape info
#                 row_dict = {
#                     'filename': file_path.name,
#                     'original_shape': str(shape_info),
#                     'total_points': data.size
#                 }
                
#                 # Instead of storing all values, calculate some summary statistics
#                 row_dict.update({
#                     'mean': np.mean(data),
#                     'std': np.std(data),
#                     'min': np.min(data),
#                     'max': np.max(data),
#                     'median': np.median(data)
#                 })
                
#                 # If you need actual mesh data points, you can add them selectively
#                 # For example, store only the first 1000 points:
#                 flattened = data.ravel()
#                 for j, val in enumerate(flattened[:1000]):
#                     row_dict[f'point_{j}'] = val
                
#                 batch_data.append(row_dict)
                
#                 # Clear memory
#                 del data
#                 del flattened
#                 gc.collect()
                
#             except Exception as e:
#                 print(f"Error processing {file_path}: {str(e)}")
#                 continue
        
#         # Create DataFrame from batch
#         df_batch = pd.DataFrame(batch_data)
        
#         # Save intermediate results if requested
#         if save_intermediate:
#             mode = 'w' if i == 0 else 'a'
#             header = i == 0
#             df_batch.to_csv(output_path, mode=mode, header=header, index=False)
#             print(f"Saved batch to {output_path}")
        
#         # If not saving intermediate results, return the final DataFrame
#         if not save_intermediate and i + batch_size >= len(npy_files):
#             return df_batch
        
#         # Clear memory
#         del df_batch
#         gc.collect()
    
#     print("Processing complete!")
    
#     if save_intermediate:
#         print(f"Final results saved to: {output_path}")
#         return pd.read_csv(output_path)

In [7]:
raw_img = np.fromfile("/home/sid/DeepView/volumes/scan_001.raw", dtype = np.dtype('<u2'))
raw_img = np.reshape(np.array(raw_img), (1280, 768, 768)).astype(np.float32)

points = dt.extract_point_data(raw_img)
vertices = dt.extract_vertices(points)
vertices

array([[287., 278., 414.],
       [287., 278., 415.],
       [287., 278., 416.],
       ...,
       [744., 391., 556.],
       [744., 392., 556.],
       [744., 393., 556.]])

In [8]:


# Define the directory containing raw files
raw_files_directory = "/home/sid/DeepView/volumes"

# Initialize an empty list to store data for DataFrame
compiled_data = []

# List all .raw files in the directory
raw_files = [file for file in os.listdir(raw_files_directory) if file.endswith(".raw")]

# Define batch size
batch_size = 10
total_files = len(raw_files)
print(f"Total files to process: {total_files}")

# Process files in batches
for i in range(0, total_files, batch_size):
    batch = raw_files[i:i + batch_size]
    print(f"Processing batch {i // batch_size + 1}/{(total_files - 1) // batch_size + 1}...")

    for file_name in batch:
        file_path = os.path.join(raw_files_directory, file_name)
        try:
            # Load and reshape raw image
            raw_img = np.fromfile(file_path, dtype=np.dtype('<u2'))
            raw_img = np.reshape(np.array(raw_img), (1280, 768, 768)).astype(np.float32)
            
            # Process the image
            points = dt.extract_point_data(raw_img)
            vertices = dt.extract_vertices(points)
            
            # Compile the data into a dictionary
            compiled_data.append({
                "file_name": file_name,
                "vertices": vertices
            })
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
    
    # Save intermediate results to avoid data loss
    temp_df = pd.DataFrame(compiled_data)
    temp_df.to_csv("compiled_mesh_data_temp.csv", index=False)
    print(f"Batch {i // batch_size + 1} completed and saved.")

# Convert compiled data into a final DataFrame
mesh_data_df = pd.DataFrame(compiled_data)

# Save the final DataFrame to a CSV file
mesh_data_df.to_csv("compiled_mesh_data.csv", index=False)

print("All files processed successfully.")


Total files to process: 87
Processing batch 1/9...
Batch 1 completed and saved.
Processing batch 2/9...
Batch 2 completed and saved.
Processing batch 3/9...


KeyboardInterrupt: 

In [11]:
mesh_arrays = pd.read_csv("compiled_mesh_data.csv")
mesh_arrays.iloc[0]

file_name                                         scan_046.raw
vertices     [[335. 411. 404.]\n [335. 411. 405.]\n [335. 4...
Name: 0, dtype: object